<a href="https://colab.research.google.com/github/eduardodacostasoares/Data_Science/blob/master/aula2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Transformando Variáveis**#

##**Bibliotecas**##

**1) Numpy**: biblioteca de álgebra linear do Python

**2) Pandas**: biblioteca de manipulação de Dataframes

**3) Pyplot**: biblioteca de visualização de dados

**4) Seaborn**: outra biblioteca de visualização de dados

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split #particionamento da base de dados em DADOS DE TREINO e DADOS DE TESTE
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression

np.set_printoptions(threshold=None,precision=2)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('precision', 2)

##**Base de dados do Titanic**##
   - Encontrada no Kaggle
   

###DESCRIÇÃO DOS ATRIBUTOS###
- **survival** = Sobrevivente (0=Não; 1=Sim)
- **pclass**   = Classe do passageiro (1 = 1ª Classe; 2 = 2ª Classe; 3 = 3ª Classe)
- **name** = Nome(str)
- **age** = idade(int)
- **sex** = Sexo (M; F)
- **sibsp** = número de irmãos/cônjuges a bordo
- **parch** = número de pais e filhos
- **ticket** = número do bilhete
- **fare** = tarifa do passageiro
- **cabin** = cabine
- **embarked** = embarque (C = Cherbourg; Q = Queenstown; S = Southampton).



###**Upload de arquivos**###

In [2]:
from google.colab import files

uploaded = files.upload()

Saving train.csv to train.csv


In [3]:
titanic_train = pd.read_csv(next(iter(uploaded.keys())))

print("\nDimensões de Titanic:\n{0}\n".format(titanic_train.shape))
print('\nCampos de Titanic:\n{0}\n'.format(list(titanic_train.keys())))
print("\nTipos de dados:\n{0}\n".format(titanic_train.dtypes))


Dimensões de Titanic:
(891, 12)


Campos de Titanic:
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


Tipos de dados:
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object



##**ESTATÍSTICA DESCRITIVA DOS DADOS**##

O comando **describe** exibe prioritariamente os campos numéricos. Deve-se isolar os campos categóricos para serem exibidos posteriormente.

In [5]:
# Exibe apenas os campos numéricos:

print(titanic_train.describe())

# Para se ter uma visão dos atributos categórios, os atributos numéricos são
# descartados.

categ = titanic_train.dtypes[titanic_train.dtypes == "object"].index

print("\n", titanic_train[categ].describe(), sep='\n')

       PassengerId  Survived  Pclass     Age   SibSp   Parch    Fare
count       891.00    891.00  891.00  714.00  891.00  891.00  891.00
mean        446.00      0.38    2.31   29.70    0.52    0.38   32.20
std         257.35      0.49    0.84   14.53    1.10    0.81   49.69
min           1.00      0.00    1.00    0.42    0.00    0.00    0.00
25%         223.50      0.00    2.00   20.12    0.00    0.00    7.91
50%         446.00      0.00    3.00   28.00    0.00    0.00   14.45
75%         668.50      1.00    3.00   38.00    1.00    0.00   31.00
max         891.00      1.00    3.00   80.00    8.00    6.00  512.33


                                   Name   Sex    Ticket        Cabin Embarked
count                               891   891       891          204      889
unique                              891     2       681          147        3
top     Goodwin, Master. Sidney Leonard  male  CA. 2343  C23 C25 C27        S
freq                                  1   577         7          

##**REMOÇÃO DE ATRIBUTOS IRRELEVANTES**##

Os atributos **survival** (objetivo ou label), e atributos que descrevem os passageiros ou os agrupam em categorias são úteis e serão mantidos, por exemplo: **Pclass**, **Sex**, **Age**, **SibSp**, **Parch**, **Fare** e **Embarked**.



*   **passengerId** é apenas uma chave primária para identificar um passageiro, e não é relevante para o problema.
*   **Name** não é útil para a previsão, mas pode ser útil para a identificação dos registros ou pós-processamento (por exemplo, extrair o último nome).
*   **Ticket** não identifica o registro e nem descreve o passageiro, por isso, deve ser removido.
*   **Cabin** não identifica bem os passageiros, mas, pode ser útil utilizarmos o padrão "Letra + número", para descrever os passageiros pelo andar do local da cabine.  

In [6]:
del titanic_train["PassengerId"]
del titanic_train["Ticket"]

# o número de atributos foi reduzido para 10

print("\nDimensões de Titanic:\n{0}\n".format(titanic_train.shape))


Dimensões de Titanic:
(891, 10)



##**TRANSFORMAÇÕES DE DADOS**##

**Converter numérico em categórico**.

Variáveis categóricas codificadas numericamente possuem baixa legibilidade. Portanto, podem ser candidatas a serem recodificadas.

In [8]:
new_survived = pd.Categorical(titanic_train["Survived"])
new_survived = new_survived.rename_categories(["Morreu", "Sobreviveu"])
titanic_train["Survived"] = new_survived

new_Pclass = pd.Categorical(titanic_train["Pclass"], ordered=True)
new_Pclass = new_Pclass.rename_categories(["1a Classe","2a Classe","3a Classe"])
titanic_train["Pclass"] = new_Pclass

print("\nTipos dos dados:\n{0}\n".format(titanic_train.dtypes))
categ = titanic_train.dtypes[titanic_train.dtypes == "category"].index
print("\n", titanic_train[categ].describe(), sep='\n')


Tipos dos dados:
Survived    category
Pclass      category
Name          object
Sex           object
Age          float64
SibSp          int64
Parch          int64
Fare         float64
Cabin         object
Embarked      object
dtype: object



       Survived     Pclass
count       891        891
unique        2          3
top      Morreu  3a Classe
freq        549        491
